<a href="https://colab.research.google.com/github/carlosks/carlosks/blob/main/Dividir_Arquivo_Desafio_Limitado_20M_cada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Conexão com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

import json

import os



# Função auxiliar para salvar parte do DataFrame no formato JSON

def save_json_chunk(df_chunk, part_num, base_path):

    file_name = f'{base_path}/TRN_Dataset_Part_{part_num}.json'

    df_chunk.to_json(file_name, orient='records', lines=True)

    return file_name



# Função para dividir e salvar o DataFrame com limite de 20MB por arquivo

def split_dataframe_to_json(df_filtered, base_path, max_file_size_mb=20):

    max_file_size_bytes = max_file_size_mb * 1024 * 1024  # Converter MB para bytes

    part_num = 1

    chunk_size = 1000  # Número de linhas processadas de cada vez

    start_idx = 0



    # Inicializa um DataFrame vazio para armazenar os dados temporariamente

    df_chunk = pd.DataFrame()



    while start_idx < len(df_filtered):

        # Adiciona um pedaço do DataFrame ao chunk

        df_chunk = pd.concat([df_chunk, df_filtered.iloc[start_idx:start_idx + chunk_size]])

        # Salva o chunk temporariamente em um arquivo para medir o tamanho

        file_name = save_json_chunk(df_chunk, part_num, base_path)



        # Verifica o tamanho do arquivo

        file_size = os.path.getsize(file_name)



        if file_size >= max_file_size_bytes:

            # Se o arquivo exceder o tamanho, começa um novo arquivo e aumenta o número da parte

            print(f"Arquivo {file_name} salvo com {file_size / (1024 * 1024):.2f} MB.")

            part_num += 1

            df_chunk = pd.DataFrame()  # Limpa o DataFrame temporário para iniciar novo arquivo



        start_idx += chunk_size



    # Salva qualquer dado restante que não tenha sido salvo

    if not df_chunk.empty:

        file_name = save_json_chunk(df_chunk, part_num, base_path)

        print(f"Arquivo {file_name} salvo com {os.path.getsize(file_name) / (1024 * 1024):.2f} MB.")



# Carregar o JSON de um arquivo ou string

with open('/content/drive/MyDrive/Desafio_Fase_3/trn.json', 'r') as f:

    file_content = f.read()

    try:

        dados_json = json.loads(file_content)

    except json.JSONDecodeError:

        dados_json = [json.loads(line) for line in file_content.splitlines() if line.strip()]



# Converter o JSON em um DataFrame normalizado

df = pd.json_normalize(dados_json)



# Filtrar registros cujo uid seja numérico

try:

    df_filtered = df[df['uid'].apply(lambda x: str(x).isdigit())]

except KeyError:

    df_filtered = df.drop(columns=['uid'], errors='ignore')



# Filtrar registros onde 'content' não está vazio

try:

    df_filtered = df_filtered[df_filtered['content'].notna() & (df_filtered['content'].str.strip() != '')]

except KeyError:

    df_filtered = df_filtered.drop(columns=['content'], errors='ignore')



# Manter apenas as colunas 'title' e 'content'

df_filtered = df_filtered[['title', 'content']]



# Dividir e salvar o DataFrame em arquivos de no máximo 20 MB

split_dataframe_to_json(df_filtered, base_path='/content/drive/MyDrive/Desafio_Fase_3', max_file_size_mb=20)

Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_1.json salvo com 20.26 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_2.json salvo com 20.56 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_3.json salvo com 20.63 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_4.json salvo com 20.60 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_5.json salvo com 20.06 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_6.json salvo com 21.10 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_7.json salvo com 20.01 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_8.json salvo com 20.64 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_9.json salvo com 20.24 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_10.json salvo com 20.37 MB.
Arquivo /content/drive/MyDrive/Desafio_Fase_3/TRN_Dataset_Part_11.json salvo com 21.01 MB.
Arquivo 